In [1]:
import pandas as pd
import numpy as np
from datetime import date
from statistics import mean, mode
from pandas.api.types import CategoricalDtype

In [2]:
today = date.today()
print(today)

2023-10-19


# Tables

In [3]:
path_tables = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 2 tables/"

In [4]:
# tables
filename_careerorglink = "careerorglink.xlsx"
filename_leadercareerlink = "leadercareerlink.xlsx"
filename_orgtree = "orgtree.xlsx"
filename_elected = "positions_elected.xlsx"

In [5]:
col = pd.read_excel(path_tables + filename_careerorglink,dtype="str")
col.shape

(9002, 12)

In [6]:
col.columns

Index(['CareerString', 'CareerDateString_2022', 'IsJob', 'MultipleSubstrings',
       'CareerStartYear', 'CareerStartMonth', 'CareerSubstring',
       'InstitutionType', 'PrimaryInstitution', 'OrgName', 'Position',
       'Notes'],
      dtype='object')

In [7]:
lcl = pd.read_excel(path_tables + filename_leadercareerlink,dtype="str")
lcl.shape

(12617, 3)

In [8]:
lcl.columns

Index(['LeaderID', 'CareerString', 'CareerDateString_2022'], dtype='object')

In [9]:
org = pd.read_excel(path_tables + filename_orgtree,dtype="str")
org.shape

(2368, 19)

In [10]:
org.columns

Index(['InstitutionType', 'OrgType', 'PrimaryInstitution', 'OrgName',
       'PI_Index', 'OrgRank', 'P1', 'P2', 'P3', 'Alias_OrgName',
       'LinkToNext_PI', 'LinkToNext_Org', 'LinkToNext_Year', 'Notes',
       'L1_Index', 'L2_Index', 'L3_Index', 'L4_Index', 'L5_Index'],
      dtype='object')

In [11]:
elected = pd.read_excel(path_tables + filename_elected,dtype="str")
elected.shape

(105, 4)

In [12]:
elected.columns

Index(['PrimaryInstitution', 'OrgName', 'Position', 'IsElected'], dtype='object')

# Queries

In [13]:
path_queries = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 3 queries/"

In [14]:
# jobs and job transitions, elected & unelected, in and out of gov
filename_leaderjob_electUnelect_inOutgov = "leaderjob_electUnelect_inOutgov.xlsx"
filename_leaderjobtransition_electUnelect_inOutgov = "leaderjobtransition_electUnelect_inOutgov.xlsx"

In [15]:
trans = pd.read_excel(path_queries + filename_leaderjobtransition_electUnelect_inOutgov,dtype="str")
trans.shape

(6311, 29)

#### change datatypes

In [129]:
trans = trans.astype({"OrgAdvance":"int","PositionAdvance":"int","CareerStartYear_1":"int"})
trans.dtypes

LeaderID                   object
CareerString_1             object
CareerDateString_2022_1    object
CareerStartYear_1           int32
CareerStartMonth_1         object
CareerStartDate_1          object
CareerSubstring_1          object
InstitutionType_1          object
PrimaryInstitution_1       object
OrgName_1                  object
Position_1                 object
IsElected_1                object
OrgRank_1                  object
PositionRank_1             object
CareerString_2             object
CareerDateString_2022_2    object
CareerStartYear_2          object
CareerStartMonth_2         object
CareerStartDate_2          object
CareerSubstring_2          object
InstitutionType_2          object
PrimaryInstitution_2       object
OrgName_2                  object
Position_2                 object
IsElected_2                object
OrgRank_2                  object
PositionRank_2             object
OrgAdvance                  int32
PositionAdvance             int32
dtype: object

# Functions

In [130]:
def merge_results(m):
    
    print("\nMerge Results...")
    print("")
    print("\tshape     :",m.shape)
    print("\tleft_only :",m[m["_merge"]=="left_only"].shape)
    print("\tboth      :",m[m["_merge"]=="both"].shape)
    print("\tright_only:",m[m["_merge"]=="right_only"].shape)

In [131]:
# using this on (PI,OrgName) will ensure unique & non-null keys
# using this on a larger df will ensure unique rows and non-null keys, but not unique keys

def unique_non_null_rows(olddf):
    
    df = olddf.copy()
    
    
    ### drop duplicates
    df = df.drop_duplicates(keep="first",ignore_index=True)

    ### drop null rows
    df.dropna(how="all",axis=0,inplace=True)
    
    # drop rows with null PI
    df = df[~(df["PrimaryInstitution"].isna())]
        
    ### drop rows where PI contains stop words
    stop_words_lower = ["uncertain","current","deprecated","please_revise"]
    df = df[ ~ df["PrimaryInstitution"].str.lower().isin(stop_words_lower)]
            

    print("\nUnique Non-null Rows...")
    print("")
    print("\tNon-unique rows:",olddf.shape)
    print("\tUnique rows    :",df.shape)

    ### sort
    df = df.sort_values(["PrimaryInstitution","OrgName"])
    
    return df

In [132]:
def create_time_series(series,group_var,count_var):
    
    yeardist = series.groupby(group_var,as_index=False).count().sort_values(group_var)
    
    max_year = max(series[group_var])
    x = pd.DataFrame({"year":pd.Series(range(min(yeardist[group_var].astype(int)),max(yeardist[group_var].astype(int))+1))})
    
    yeardist[group_var] = yeardist[group_var].astype(int,errors="ignore")
    
    ts = x.merge(yeardist,left_on="year",right_on=group_var,how="left")
    ts.loc[ts[count_var].isna(),count_var]=0
    # merge_results(ts)
    
    ts_columns = ["year",count_var]
    ts = ts[ts_columns]
    
    return ts

# Analysis - Research Note

In [29]:
path_analysis = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/analysis/"

In [30]:
# analysis sub-paths
study0_path = "2023.10.04 Study 0 - research note/"
study1_path = "2023.10.04 Study 1 - political capital/"
study2_path = "2023.10.04 Study 2 - commitment vs control/"
study3_path = "2023.10.04 Study 3 - reds vs experts/"

#### unique SPA orgs and positions

In [17]:
col.head(5)

,CareerString,CareerDateString_2022,IsJob,MultipleSubstrings,CareerStartYear,CareerStartMonth,CareerSubstring,InstitutionType,PrimaryInstitution,OrgName,Position,Notes
0,4.15문학창작단 단장,1989.04,True,1,1989,4,NaN,당외곽및사회단체_사회부문(별책),4.15문화창작단,NaN,단장,NaN
1,"1989. 4.15문학창작단 단장, 조선작가동맹 통일문학담당 부위원장",NaN,True,2,1989,4,4.15 문화창작단 단장,당외곽및사회단체_사회부문(별책),4.15문화창작단,NaN,단장,NaN
2,2004. 4.15문학창작단 부단장,NaN,True,1,2004,NaN,NaN,당외곽및사회단체_사회부문(별책),4.15문화창작단,NaN,부단장,NaN
3,4.15문학창작단 부단장,2004.04,True,1,2004,4,NaN,당외곽및사회단체_사회부문(별책),4.15문화창작단,NaN,부단장,NaN
4,7.7연합기업소 기사장,1989.12,True,1,1989,12,NaN,UNCERTAIN,7.7연합기업소,NaN,기사장,NaN


In [28]:
spa_columns = ["InstitutionType","PrimaryInstitution","OrgName","Position"]
spa_orgpositions = col.loc[col.PrimaryInstitution=="최고인민회의",spa_columns].drop_duplicates()

In [31]:
spa_fname = "spa org and positions.xlsx"
spa_orgpositions.to_excel(path_analysis + study0_path + spa_fname,index=False)

### leaders with SPA - use leaderjobtransition_electUnelect_inOutgov which contains elected jobs

In [135]:
trans.head()

,LeaderID,CareerString_1,CareerDateString_2022_1,CareerStartYear_1,CareerStartMonth_1,CareerStartDate_1,CareerSubstring_1,InstitutionType_1,PrimaryInstitution_1,OrgName_1,...,CareerSubstring_2,InstitutionType_2,PrimaryInstitution_2,OrgName_2,Position_2,IsElected_2,OrgRank_2,PositionRank_2,OrgAdvance,PositionAdvance
0,리원일,1997.11 노동행정부 부부장,NaN,1997,11,199711,NaN,정권기관,정무원,노동행정부,...,NaN,정권기관,내각,노동성,상,NaN,1,1,0,1
1,리원일,1997.11 노동행정부 부부장,NaN,1997,11,199711,NaN,정권기관,정무원,노동행정부,...,NaN,정권기관,최고인민회의,NaN,대의원,1,0,3,1,-1
2,리원일,노동성 상 *후임 : 강영수,1997.11,1997,11,199711,NaN,정권기관,내각,노동성,...,NaN,정권기관,내각,노동성,상,NaN,1,1,0,0
3,리원일,노동성 상 *후임 : 강영수,1997.11,1997,11,199711,NaN,정권기관,내각,노동성,...,NaN,정권기관,최고인민회의,NaN,대의원,1,0,3,1,-2
4,리원일,1998. 9 노동상,NaN,1998,9,199809,NaN,정권기관,내각,노동성,...,NaN,국제친선단체,북-이란 친선협회,NaN,위원장,NaN,0,1,1,0


In [136]:
# trans.LeaderID.unique()

In [145]:
# number of elites who have records before taking a job in the 최고인민회의, elected or unelected
trans.loc[(trans.PrimaryInstitution_2=="최고인민회의"),"LeaderID"].unique().shape

(428,)

In [146]:
# number of elites who have records before taking an elected job in the 최고인민회의
trans.loc[(trans.PrimaryInstitution_2=="최고인민회의") & (trans.IsElected_2=="1"),"LeaderID"].unique().shape

(424,)

In [143]:
# number of elites who have records after taking a job in the 최고인민회의, elected or unelected
trans.loc[(trans.PrimaryInstitution_1=="최고인민회의"),"LeaderID"].unique().shape

(402,)

In [144]:
# number of elites who have records after taking an elected job in the 최고인민회의
trans.loc[(trans.PrimaryInstitution_1=="최고인민회의") & (trans.IsElected_1=="1"),"LeaderID"].unique().shape

(393,)

In [141]:
trans.LeaderID.unique().shape

(539,)

In [142]:
trans.columns

Index(['LeaderID', 'CareerString_1', 'CareerDateString_2022_1',
       'CareerStartYear_1', 'CareerStartMonth_1', 'CareerStartDate_1',
       'CareerSubstring_1', 'InstitutionType_1', 'PrimaryInstitution_1',
       'OrgName_1', 'Position_1', 'IsElected_1', 'OrgRank_1', 'PositionRank_1',
       'CareerString_2', 'CareerDateString_2022_2', 'CareerStartYear_2',
       'CareerStartMonth_2', 'CareerStartDate_2', 'CareerSubstring_2',
       'InstitutionType_2', 'PrimaryInstitution_2', 'OrgName_2', 'Position_2',
       'IsElected_2', 'OrgRank_2', 'PositionRank_2', 'OrgAdvance',
       'PositionAdvance'],
      dtype='object')